In [1]:
import ipywidgets as widgets
from IPython.display import display, Image
import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown


# Function to get the binomial name for an animal from Wikipedia or ITIS
def get_binomial_name(animal_name):
    # Try to get the binomial name from Wikipedia
    base_url_wiki = "https://en.wikipedia.org/wiki/"
    response_wiki = requests.get(base_url_wiki + animal_name)
    if response_wiki.status_code == 200:
        soup_wiki = BeautifulSoup(response_wiki.content, "html.parser")
        binomial_name_element_wiki = soup_wiki.find("span", class_="binomial")
        if binomial_name_element_wiki:
            binomial_name_wiki = binomial_name_element_wiki.text.strip()
            return binomial_name_wiki

    # If not found on Wikipedia, try to get it from ITIS
    base_url_itis = f"https://www.itis.gov/servlet/SingleRpt/SingleRpt?search_topic=TSN&search_value={animal_name}"
    response_itis = requests.get(base_url_itis)
    if response_itis.status_code == 200:
        soup_itis = BeautifulSoup(response_itis.content, "html.parser")
        name_element_itis = soup_itis.find("td", attrs={"class": "contentTaxonName"})
        if name_element_itis:
            binomial_name_itis = name_element_itis.find("em")
            if binomial_name_itis:
                return binomial_name_itis.text.strip()

    return "Not Available"

# Function to get the conservation status from the Wikipedia page
def get_conservation_status(soup):
    # Find the infobox table on the page
    infobox = soup.find("table", class_="infobox")

    if infobox:
        # Find all table rows in the infobox
        rows = infobox.find_all("tr")

        for row in rows:
            # Check if the row contains the term "Conservation status"
            if "Conservation status" in row.get_text():
                # Extract the text from the row
                row_text = row.get_text().strip()

                # Check if the text contains parentheses (some conservation statuses are in parentheses)
                if "(" in row_text and ")" in row_text:
                    # Extract the text inside parentheses as the conservation status
                    conservation_status = row_text[row_text.index("(") + 1:row_text.index(")")]
                else:
                    # If no parentheses, use the entire row text as the conservation status
                    conservation_status = row_text

                return conservation_status

    return "Not Available"


# Function to get information about an animal from Wikipedia
def get_animal_info(animal_name):
    base_url = "https://en.wikipedia.org/wiki/"
    response = requests.get(base_url + animal_name)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, "html.parser")
        common_name_element = soup.find("h1", class_="firstHeading")
        if common_name_element:
            common_name = common_name_element.text.strip()
            # Get the binomial name for the animal
            binomial_name = get_binomial_name(animal_name)
            # Get the conservation status from the Wikipedia infobox table
            conservation_status = get_conservation_status(soup)

            return {
                "Common Name": common_name,
                "Scientific Name": binomial_name,
                "Conservation Status": conservation_status
            }
        else:
            return {"error": "Animal not found on Wikipedia."}
    else:
        return {"error": "Failed to fetch data from Wikipedia."}

# Function to get the image URL for the animal from Wikipedia
def get_animal_image_url(animal_name):
    base_url = "https://en.wikipedia.org/api/rest_v1/page/summary/"
    url = f"{base_url}{animal_name}"
    response = requests.get(url)
    data = response.json()
    if "thumbnail" in data:
        # Extract the image URL
        image_url = data["thumbnail"]["source"]
        return image_url
    else:
        return None

# Function to display the animal information and image
def search_animal_info(btn):
    animal_name = text_input.value.strip()
    animal_info = get_animal_info(animal_name)

    if "error" in animal_info:
        output_text.value = animal_info["error"]
        img_widget.clear_output()
    else:
        output_text.value = "Information for " + animal_name
        for key, value in animal_info.items():
            output_text.value += f"\n{key}: {value}"

        image_url = get_animal_image_url(animal_name)
        if image_url is not None:
            with img_widget:
                img_widget.clear_output(wait=True)
                img = Image(url=image_url)
                display(img)
        else:
            img_widget.clear_output()

            
# Function to generate the link for adoption based on the animal name
def get_adoption_link(animal_name):
    # Check if the animal name starts with a consonant or a vowel
    first_letter = animal_name[0].lower()
    if first_letter in ['a', 'e', 'i', 'o', 'u']:
        link_url = f"https://support.wwf.org.uk/adopt-an-{animal_name.lower()}"
    else:
        link_url = f"https://support.wwf.org.uk/adopt-a-{animal_name.lower()}"
    return link_url

# Function to display the animal information and image, along with the clickable link for adoption
def get_adoption_info(btn):
    animal_name = text_input.value.strip()
    animal_info = get_animal_info(animal_name)

    if "error" in animal_info:
        output_text.value = animal_info["error"]
        img_widget.clear_output()
    else:
        output_text.value = "Information for " + animal_name
        for key, value in animal_info.items():
            output_text.value += f"\n{key}: {value}"

        # Add the clickable link to the WWF website for adopting the animal
        link_url = get_adoption_link(animal_name)
        adopt_link = f"[Adopt Now]({link_url})"
        display(Markdown(adopt_link))

        image_url = get_animal_image_url(animal_name)
        if image_url is not None:
            with img_widget:
                img_widget.clear_output(wait=True)
                img = Image(url=image_url)
                display(img)
        else:
            img_widget.clear_output()
            
            
            
# Create widgets
text_input = widgets.Text(placeholder="Enter the name of an animal")
search_button = widgets.Button(description="Search")
search_button.on_click(get_adoption_info)
output_text = widgets.Textarea(disabled=True)
img_widget = widgets.Output()

# Display widgets
display(text_input, search_button, output_text, img_widget)


Text(value='', placeholder='Enter the name of an animal')

Button(description='Search', style=ButtonStyle())

Textarea(value='', disabled=True)

Output()